In [2]:
import pandas as pd

In [3]:
# Collecting the database

# Change the file path if going to test!!
df_case = pd.read_excel("files/Data_base_Vendas(1).xlsx", sheet_name="Database")

Treating the data and collecting some information

In [4]:
top_selling_by_qty = df_case.set_index("Customer")[['Material Name', 'Delivered qty. in KG']].groupby(['Material Name']).sum()

In [5]:
top_10_selling_by_qty = top_selling_by_qty.sort_values(by="Delivered qty. in KG", ascending=False).head(10)

In [6]:
top_selling_by_revenue = df_case.set_index("Customer")[['Material Name',"Revenue USD"]].groupby(['Material Name']).sum()

In [7]:
top_10_selling_by_revenue = top_selling_by_revenue.sort_values(by="Revenue USD", ascending=False).head(10)

In [8]:
highest_plants = df_case[["Plant", "Delivered qty. in KG"]].groupby("Plant").sum()

In [9]:
highest_plants.sort_values(by="Delivered qty. in KG", ascending=False, inplace=True)

In [10]:
biggest_buyers = df_case[["Customer", "Revenue USD"]].groupby(by="Customer").sum()
biggest_buyers.sort_values(by="Revenue USD", ascending=False, inplace=True)

In [11]:
top_10_biggest_buyers = biggest_buyers.head(10)

In [12]:
df_case['New actual date'] = pd.to_datetime(df_case['Actual delivery date'])

In [13]:
df_case['New Requested date'] = pd.to_datetime(df_case['Requested delivery date'], errors='coerce')

In [14]:
df_case['Days to deliver'] = (df_case['New actual date'] - df_case['New Requested date']).dt.days

In [15]:
mean_days_to_deliver = df_case['Days to deliver'].mean()

In [16]:
reason_series = df_case['Reason for delay']
reason_series = reason_series.mask(reason_series == 'OnTime', other=1)
reason_series = reason_series.where(reason_series == 1, other=0)

In [17]:
OnTime_rate = reason_series.sum()*100/93405

In [18]:
OnTime_rate

90.213585996467

Writing the data into the excel file

In [19]:
# Change the file path to test!!

with pd.ExcelWriter(
    "files/Database_Vendas.xlsx",
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
) as writer:
    top_10_selling_by_qty.to_excel(writer, sheet_name="top_10_selling_by_qty")
    top_10_selling_by_revenue.to_excel(writer, sheet_name="top_10_selling_by_revenue")
    highest_plants.to_excel(writer, sheet_name="highest_plants")
    top_10_biggest_buyers.to_excel(writer, sheet_name="top_10_biggest_buyers")

Some more analysis

In [20]:
def participation(series):
    dic = {'total': 0}
    for key, value in series.items():
        dic[key] = dic.get(key, 0) + value
        dic['total']+=value

    for key in dic.keys():
        if key == 'total':
            pass
        else:
            dic[key] = dic[key]*100/dic['total']
    return dic

In [21]:
#Participation by country
country_series = df_case.copy(deep=True).set_index('Country')
country_series = country_series['Revenue USD']
participation_dic = participation(country_series)
participation_df = pd.DataFrame.from_dict(participation_dic, orient='index')

#Participation by Division
division_series = df_case.copy(deep=True).set_index('Division')
division_series = division_series['Revenue USD']
division_dic = participation(division_series)

Writing it into the excel file - again...

In [22]:
# Change the file path to test!!

with pd.ExcelWriter(
    "files/Database_Vendas.xlsx",
    mode="a",
    engine="openpyxl",
    if_sheet_exists="replace",
) as writer:
    participation_df.to_excel(writer, sheet_name="country_participation")